In [14]:
import re

In [15]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
import io
import numpy as np
import os
from zipfile import ZipFile
import warnings

!pip install fuzzywuzzy
import fuzzywuzzy

from fuzzywuzzy import fuzz
import re

warnings.filterwarnings("ignore")
import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
from fuzzywuzzy import process

In [ ]:
!wget https://zenodo.org/record/7441645/files/dwug_de.zip
with ZipFile('dwug_de.zip', 'r') as dwug_de:
    dwug_de.extractall()

In [ ]:
!wget http://lcl.uniroma1.it/wsdeval/data/WSD_Unified_Evaluation_Datasets.zip
with ZipFile('WSD_Unified_Evaluation_Datasets.zip', 'r') as dta:
    dta.extractall()

In [5]:
import xml.etree.ElementTree as ET
tree = ET.parse('/content/WSD_Unified_Evaluation_Datasets/ALL/ALL.data.xml')
root = tree.getroot()

In [6]:
lemma_sentence_dict = {}
for text in root.iter('text'):
  #print(text.attrib) text ID
    lemma_sentences = []
    surface_sentences = []
    for indd in range(len(text)): 
      sentences = []
      if indd == 0:
        sentences.append(text[indd])
        sentences.append(text[indd+1]) 
      elif indd == len(text)-1:
        sentences.append(text[indd-1])
        sentences.append(text[indd])
      else:
        sentences.append(text[indd-1]) #Sentence-1 , Sentence, Sentence +1     
        sentences.append(text[indd])
        sentences.append(text[indd+1])
      temp_lemma_sentences = []
      temp_surface_sentences = []
      index_targetword = []
      wordid_list = []
      pos = []
      for sentence in sentences:
        sentenceid = sentence.attrib['id']
        i = 0
        lemma_sentence = []
        surface_sentence = []

        for word in sentence:
          lemma = word.attrib['lemma']
          surface = word.text

          if indd == 0:
            if sentences.index(sentence) == 0:
              p = word.attrib['pos']
              try:
                wordid = word.attrib['id']
                index_targetword.append(i)
                
              except KeyError:
                wordid = "NA"
              wordid_list.append(wordid)
              pos.append(p)
          else:
            if sentences.index(sentence) == 1:
              p = word.attrib['pos']
              try:
                wordid = word.attrib['id']
                index_targetword.append(i)
                
              except KeyError:
                wordid = "NA"
              wordid_list.append(wordid)
              pos.append(p)
          surface_sentence.append(surface)
          lemma_sentence.append(lemma)  

          i += 1
        temp_lemma_sentences.append(lemma_sentence)
        temp_surface_sentences.append(surface_sentence)
        assert len(surface_sentence) == len(lemma_sentence) 
        

      surface_sentences.append(temp_surface_sentences)   #[[s1, s2, s3], ]
      lemma_sentences.append(temp_lemma_sentences)
      #print(index_targetword)

      a = 0
      for ind in index_targetword:
        if indd == 0:
          try:
            lemma_sentence_dict[temp_lemma_sentences[0][ind]].append((
              " ".join([' '.join(x) for x in temp_lemma_sentences]), 
              " ".join([' '.join(x) for x in temp_surface_sentences]),
              str(ind),
              str(ind),
              temp_lemma_sentences[0][ind], str(0)+':'+str(len(' '.join(temp_surface_sentences[0]))), str(0)+':'+str(len(temp_surface_sentences[0])),
               #temp_lemma_sentences[0][ind], #str(list(nlp(" ".join([' '.join(x) for x in temp_surface_sentences])).sents)[0]),
              wordid_list[ind] , pos[ind],         
              ))
          except KeyError:
            lemma_sentence_dict[temp_lemma_sentences[0][ind]] = [(
              " ".join([' '.join(x) for x in temp_lemma_sentences]), 
              " ".join([' '.join(x) for x in temp_surface_sentences]),
              str(ind),
              str(ind),
              temp_lemma_sentences[0][ind], str(0)+':'+str(len(' '.join(temp_surface_sentences[0]))), str(0)+':'+str(len(temp_surface_sentences[0])),
               #temp_lemma_sentences[0][ind], #str(list(nlp(" ".join([' '.join(x) for x in temp_surface_sentences])).sents)[0]),
              wordid_list[ind] , pos[ind]         
            )]
        else:
            try:
                  lemma_sentence_dict[temp_lemma_sentences[1][ind]].append((
                    " ".join([' '.join(x) for x in temp_lemma_sentences]), 
                    " ".join([' '.join(x) for x in temp_surface_sentences]),
                    str(ind),
                    str(ind),
                    temp_lemma_sentences[1][ind], str(len(' '.join(temp_surface_sentences[0])))+':'+str(len(' '.join(temp_surface_sentences[0]))+len(' '.join(temp_surface_sentences[1]))+1),
                    str(len((temp_surface_sentences[0])))+':'+str(len((temp_surface_sentences[0]))+len((temp_surface_sentences[1]))),
                    #temp_lemma_sentences[1][ind],# str((" ".join([' '.join(x) for x in temp_surface_sentences]))),
                    wordid_list[ind] , pos[ind]       
                    ))
            except KeyError:
                  lemma_sentence_dict[temp_lemma_sentences[1][ind]] = [(
                    " ".join([' '.join(x) for x in temp_lemma_sentences]), 
                    " ".join([' '.join(x) for x in temp_surface_sentences]),
                    str(ind),
                    str(ind),
                    temp_lemma_sentences[1][ind], str(len(' '.join(temp_surface_sentences[0])))+':'+str(len(' '.join(temp_surface_sentences[0]))+len(' '.join(temp_surface_sentences[1]))+1), 
                    str(len((temp_surface_sentences[0])))+':'+str(len((temp_surface_sentences[0]))+len((temp_surface_sentences[1]))),
                    #temp_lemma_sentences[1][ind], #str(0)+ ':'+str((" ".join([' '.join(x) for x in temp_surface_sentences]))),
                    wordid_list[ind] , pos[ind]         
                  )]
        
   

In [7]:
if not os.path.exists("data_files"):
  os.mkdir("data_files")

with open(f"data_files/all.tsv", "w") as f:
  for lemma in lemma_sentence_dict:
    #f.write(lemma +"\n")
    for x in lemma_sentence_dict[lemma]:
      to_write =  "\t".join(x)+"\n" 
      f.write(to_write)

In [8]:
def getname(dataset):
  return dataset.split('.')[0]

In [9]:
wsd = pd.read_csv('/content/data_files/all.tsv', delimiter = '\t', names = ['sent1','sent2', 'index1', 'index2', 'lemma', 'l1', 'indexes_target_sentence_tokenized','identifier', 'pos'], quoting = 3)
wsd['dataset'] = wsd['identifier'].apply(getname)
wsd = wsd[['sent2', 'lemma', 'l1', 'indexes_target_sentence_tokenized', 'identifier', 'pos', 'dataset', 'index2']]

In [11]:
wsd.rename(columns = {'sent2':'context_tokenized'}, inplace = True)
wsd.rename(columns = {'l1':'indexes_target_sentence'}, inplace = True)
wsd['context'] = wsd['context_tokenized']

In [37]:
def get_target(sentence, word):
    grams = re.split(r"[^a-zA-Z0-9]", word)
    tok = re.split(r"[^a-zA-Z0-9]", sentence)
    big = list(ngrams(tok, len(grams)))
    #print(big)
    match_ratio = 0
    gram = ""
    gram1 = ""
    idx = -1
    for index, i in enumerate(big):
        ratio = fuzz.token_sort_ratio(" ".join(i).lower(), " ".join(grams).lower())
        if ratio>match_ratio:
            match_ratio = ratio
            gram = " ".join(i).lower()
            gram1 = " ".join(grams).lower()
            idx = index
    #print(gram, gram1, match_ratio, idx)
    length = len(gram)
    length1 = len(" ".join(tok[:idx]))+1
    targ_token = sentence[length1:length1+length]
    print(sentence[length1:length1+length], length1, idx)
    if len(targ_token.split(" "))>1:
      return str(idx) + ':' + str(idx +len(targ_token.split(" ")))
    else:
      return idx
    
    #if

In [48]:
def get_indice(sentence, word):
    grams = re.split(r"[^a-zA-Z0-9]", word)
    tok = re.split(r"[^a-zA-Z0-9]", sentence)
    big = list(ngrams(tok, len(grams)))
    #print(big)
    match_ratio = 0
    gram = ""
    gram1 = ""
    idx = -1
    for index, i in enumerate(big):
        ratio = fuzz.token_sort_ratio(" ".join(i).lower(), " ".join(grams).lower())
        if ratio>match_ratio:
            match_ratio = ratio
            gram = " ".join(i).lower()
            gram1 = " ".join(grams).lower()
            idx = index
    #print(gram, gram1, match_ratio, idx)
    length = len(gram)
    length1 = len(" ".join(tok[:idx]))+1
    targ_token = sentence[length1:length1+length]
    target_indice = str(length1) + ':' + str((length1) + (length))
    return target_indice

In [52]:
wsd = wsd.reset_index(drop= True)

In [33]:
wsd['indexes_target_token_tokenized'] = wsd.apply(lambda x: get_target(x.context_tokenized, x.lemma), axis=1)

In [51]:
wsd['indexes_target_token'] = wsd.apply(lambda x: get_indice(x.context_tokenized, x.lemma), axis = 1)

In [53]:
wsd['grouping'] = 1
wsd['date'] =  ' '
wsd['description'] = ' '
wsd = wsd[['lemma', 'pos', 'date', 'grouping','identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized' ,'dataset']]


In [54]:
uses = wsd

In [ ]:
uses

In [ ]:
#need to modify the use2normalize script for token spans in wsd

In [ ]:
!git clone https://github.com/Garrafao/WUGs.git


In [ ]:
os.makedirs('/content/WUGs/scripts/misc/data/dwug_en/data/*/', exist_ok=True )

In [ ]:
import csv
uses.to_csv('/content/WUGs/scripts/misc/data/dwug_en/data/*/uses.csv', sep='\t', encoding='utf-8', quoting = csv.QUOTE_NONE, quotechar='')

In [ ]:
!cd /content/WUGs/scripts/misc && bash -e /content/WUGs/scripts/misc/use2normalize.sh 

In [ ]:
%run /content/WUGs/scripts/misc/use2normalize.py /content/WUGs/scripts/misc/data/dwug_en/data/*/uses.csv dwug_en /content/WUGs/scripts/misc/data/dwug_en/data/*/

In [ ]:
final_uses = pd.read_csv('/content/WUGs/scripts/misc/data/dwug_en/use_data/normalize/*/uses.csv', delimiter = '\t',quoting =3)

In [ ]:
final_uses

In [ ]:
wsd

In [ ]:
final_uses['context_tokenized'] = wsd['context_tokenized']
final_uses['pos'] = wsd['pos']
final_uses['dataset'] = wsd['dataset_name']
final_uses['grouping'] = wsd['grouping']

In [ ]:
labels = pd.read_csv("/content/WSD_Unified_Evaluation_Datasets/ALL/ALL.gold.key.txt", delimiter = '\t', names = ['identifier', 'identifier_sense'])
labels['identifier_sense'] = labels['identifier'].apply(lambda x: tok(x)[1])
labels['identifier'] = labels['identifier'].apply(lambda x: tok(x)[0])

In [ ]:
labels

In [ ]:
dirlist = os.listdir('dwug_de/misc/dwug_de_sense/data')

In [ ]:
lis_uses = []
for i in dirlist:
  lis_uses.append("dwug_de/misc/dwug_de_sense/data/" + i + "/uses.csv" )

In [ ]:
lis_judgments = []
for i in dirlist:
  lis_judgments.append("dwug_de/misc/dwug_de_sense/data/" + i + "/maj_3/judgments.csv" )

In [ ]:
lis_senses = []
for i in dirlist:
  lis_senses.append("dwug_de/misc/dwug_de_sense/data/" + i + "/senses.csv" )

In [ ]:
dwugde_senses = pd.DataFrame()             
for i in lis_senses:
   Tmp = pd.read_csv(i, delimiter='\t', quoting = 3)
   Tmp['dataset_name'] = i.split('/')[2]
   dwugde_senses = pd.concat([dwugde_senses, Tmp])

In [ ]:
dwugdesense_gold = pd.DataFrame()             
for i in lis_judgments:
   Tmp = pd.read_csv(i, delimiter='\t', quoting = 3)
   Tmp['dataset_name'] = i.split('/')[2]
   dwugdesense_gold = pd.concat([dwugdesense_gold, Tmp])

In [ ]:
dwugde_uses = pd.DataFrame()             
for i in lis_uses:
   Tmp = pd.read_csv(i, delimiter='\t', quoting = 3)
   Tmp['dataset_name'] = i.split('/')[2]
   dwugde_uses = pd.concat([dwugde_uses, Tmp])

In [ ]:
dwugde_uses

In [ ]:
dwugde_senses

In [ ]:
dwugdesense_gold

In [ ]:
#dwugde_judgments #should be renamed as gold and should take from maj3 folder from every lemma

In [ ]:
wsd_final_uses = pd.DataFrame()
wsd_final_uses = pd.concat([wsd,dwugde_uses], axis = 0)

In [ ]:
wsd_final_uses = wsd_final_uses.reset_index(drop = True)

In [ ]:
#dwug_de_sense, WSsim, semseval, semeval2010/2012, etc 